# 設置環境並載入套件

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute')
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute


In [2]:
import pandas as pd
import re
import unicodedata
import pickle
from collections import Counter

In [3]:
# 載入事先寫好的 functions
from compute_parameters import *

# 讀入語料

In [4]:
# 讀入 EDsplit txt 檔
edsplit_path = '../CorpusforDDL/EDsplit-1File1Days-DeleteRepeatPattern-TitleWithoutPeriod-out'
all_files = os.listdir(edsplit_path)[:10]

edsplit_corpus = []
for f in all_files:
  with open(f'{edsplit_path}/{f}', 'rb') as f:
      lines = f.readlines()
      edsplit_corpus.append(lines)

In [5]:
edsplit_corpus_decoded = []

for text in edsplit_corpus:
  text_decoded = []
  not_decoded = []
  for string in text:
    try:
      text = string.decode('cp950')
      text_decoded.append(text)
    except:
      not_decoded.append(string)
  edsplit_corpus_decoded.append(text_decoded)

In [9]:
# 前處理
def preprocess_edsplit(string):

  clean_string = unicodedata.normalize('NFKC', string) # 全形轉半形
  clean_string = re.sub(r'\r|\n', '', clean_string) # 移除換行符號
  clean_string = re.sub(r'\[\+[A-z0-9]+\]', '', clean_string) # 移除特徵標記
  clean_string = re.sub(r'\(\w+CATEGORY\)', '', clean_string) # 移除標點符號標記

  clean_string = clean_string.translate(str.maketrans({',': '，', 
                                                       '!': '！', 
                                                       '?': '？'})) # ，！？改回全形
  clean_string = clean_string.strip()
  return clean_string

In [10]:
edsplit_corpus_preprocessed = []

for text in edsplit_corpus_decoded:
  preprocessed = [preprocess_edsplit(string) for string in text]
  joined = ''.join(preprocessed)
  split_1 = re.split(r'(?<=。」|！」|？」)', joined) # 先用 。」 ！」 ？」 分隔
  split_2 = [re.split(r'(?<=[。！？])(?!」)', x) for x in split_1] # 再用 。！？ 分隔
  split = [item for sublist in split_2 for item in sublist] # 將 list of list 攤平

  edsplit_corpus_preprocessed.append(split)

In [12]:
edsplit_corpus_preprocessed[0][0]

'連接線(Na) 材廠(Nc) 報佳音(VA) 。'

## 語料格式準備 - 1
斷詞 + pos tag

In [13]:
edsplit_ws_pos = edsplit_corpus_preprocessed[0] # 先取一個檔案當例子
edsplit_ws_pos[0]

'連接線(Na) 材廠(Nc) 報佳音(VA) 。'

## 語料格式準備 - 2
斷好詞，無標點符號

In [14]:
def edsplit_get_sentence(sentence):

  sentence_words = re.sub(r'\([A-z0-9]+\)', '', sentence)
  sentence_words = re.sub(r'[^\w\s]', '', sentence_words).strip().split(' ')
  sentence_words = list(filter(None, sentence_words))

  return sentence_words

In [16]:
edsplit_ws = [edsplit_get_sentence(sentence) for sentence in edsplit_ws_pos]
edsplit_ws[0]

['連接線', '材廠', '報佳音']

## 語料格式準備 - 3
完整句子，含標點符號

In [17]:
edsplit_text = []
for sent in edsplit_ws_pos:
  res = re.sub(r'\([A-z0-9_]+\)', '', sent)
  res = res.replace(' ', '')
  edsplit_text.append(res)

edsplit_text[0]

'連接線材廠報佳音。'

# 句子長度

In [18]:
sentence_length = [len(sentence) for sentence in edsplit_ws]
sentence_length[:10]

[3, 44, 46, 46, 37, 44, 8, 38, 5, 6]

# 高頻詞與低頻詞

In [19]:
high_low_freq = [get_high_low_freq(sentence) for sentence in edsplit_ws]
high_low_freq[:10]

[{'High': 1, 'Low': 2},
 {'High': 36, 'Low': 8},
 {'High': 40, 'Low': 6},
 {'Low': 12, 'High': 34},
 {'Low': 5, 'High': 32},
 {'High': 37, 'Low': 7},
 {'Low': 4, 'High': 4},
 {'Low': 6, 'High': 32},
 {'Low': 2, 'High': 3},
 {'High': 6}]

# 詞頻

In [20]:
word_freq = [get_word_freq(sentence) for sentence in edsplit_ws]
word_freq[0]

'連接線(107) 材廠(Unknown) 報佳音(36)'

# 詞彙等級

In [21]:
word_level = [get_word_level(sentence) for sentence in edsplit_ws]
word_level[0]

{'Unknown': 3}

# 詞彙長度

In [22]:
long_word_count = [get_long_word_count(sentence) for sentence in edsplit_ws]
long_word_count[:10]

[2, 8, 4, 11, 6, 3, 0, 2, 1, 0]

# 完整的句子

In [23]:
is_complete_sentence = [get_complete_sentence(sentence) for sentence in edsplit_ws_pos]
is_complete_sentence[:10]

['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y']

# 完整的語境

In [24]:
is_complete_context = [get_complete_context(x) for x in edsplit_ws_pos]
is_complete_context[:10]

['N', 'N', 'N', 'Y', 'N', 'Y', 'N', 'Y', 'N', 'N']

# Black list

In [25]:
is_blacklist = [get_blacklist(x) for x in edsplit_text]
is_blacklist[:10]

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'Y', 'N']

# Grey list

In [26]:
is_greylist = [get_greylist(x) for x in edsplit_ws_pos]
is_greylist[:10] 

['N', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N']

# Make dataframe

In [27]:
edsplit_df = pd.DataFrame({'sentence': edsplit_text,
                        'sentence_preprocessed': [' '.join(sent) for sent in edsplit_ws],
                        'sentence_length': sentence_length,
                        'word_freq': word_freq,
                        'high_low_freq': high_low_freq,
                        'word_level': word_level,
                        'long_word_count': long_word_count,
                        'is_complete_sentence': is_complete_sentence,
                        'is_complete_context': is_complete_context,
                        'is_blacklist': is_blacklist,
                        'is_greylist': is_greylist})

In [28]:
edsplit_df = edsplit_df[edsplit_df['sentence_length']>0]
edsplit_df

,sentence,sentence_preprocessed,sentence_length,word_freq,high_low_freq,word_level,long_word_count,is_complete_sentence,is_complete_context,is_blacklist,is_greylist
0,連接線材廠報佳音。,連接線 材廠 報佳音,3,連接線(107) 材廠(Unknown) 報佳音(36),"{'High': 1, 'Low': 2}",{'Unknown': 3},2,Y,N,N,N
1,連接線材廠營運傳佳音，萬泰科在接獲俄羅斯訂單後，3月營收以2‧13億元創下單月歷史新高，第一...,連接線 材廠 營運 傳 佳音 萬泰科 在 接獲 俄羅斯 訂單 後 3月 營收 以 2 13億...,44,連接線(107) 材廠(Unknown) 營運(5641) 傳(17862) 佳音(160)...,"{'High': 36, 'Low': 8}","{'Unknown': 21, '第6級': 4, '第3*級': 1, '第1*級': 2...",8,Y,N,N,N
2,去年銅價飆漲，連接線廠紛紛關門，使得產業出現洗牌效應，訂單有向大廠集中的現象，這從網路線大廠...,去年 銅價 飆漲 連接線廠 紛紛 關門 使得 產業 出現 洗牌 效應 訂單 有 向 大廠 集...,46,去年(9462) 銅價(44) 飆漲(283) 連接線廠(Unknown) 紛紛(14617...,"{'High': 40, 'Low': 6}","{'第1級': 10, 'Unknown': 16, '第5級': 8, '第4級': 3,...",4,Y,N,N,N
3,萬泰科3月營收2‧13億元，創下單月歷史新高，萬泰科協理楊金坤表示，主要是萬泰科陸續接到美國...,萬泰科 3月 營收 2 13億 元 創下 單月 歷史 新高 萬泰科 協理 楊金坤 表示 主要...,46,萬泰科(Unknown) 3月(3) 營收(1572) 2(66480) 13億(98) 元...,"{'Low': 12, 'High': 34}","{'Unknown': 22, '第1級': 10, '第3級': 2, '第4*級': 2...",11,Y,Y,N,Y
4,萬泰科預估第一季稅前盈餘至少4，500萬元，較去年同期大成長95%，每股稅前盈餘0‧5元，預...,萬泰科 預估 第一 季 稅前盈餘 至少 4 500萬 元 較 去年 同 期 大 成長 95 ...,37,萬泰科(Unknown) 預估(1387) 第一(172544) 季(8122) 稅前盈餘(...,"{'Low': 5, 'High': 32}","{'Unknown': 18, '第6級': 4, '第2級': 2, '第3級': 1, ...",6,Y,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...
911,汽車產業大未來。,汽車 產業 大 未來,4,汽車(21028) 產業(29756) 大(525780) 未來(54445),{'High': 4},"{'第2級': 1, '第5級': 1, '第1級': 1, '第4級': 1}",0,Y,N,N,N
912,編著出版:財訊。,編著 出版 財訊,3,編著(326) 出版(37283) 財訊(83),"{'High': 2, 'Low': 1}","{'Unknown': 2, '第4*級': 1}",0,Y,N,N,Y
913,很少產業像汽車產業那樣具有獨特性，歷經百餘年的演變、全球化浪潮的衝擊，仍保有因地制宜的在地性...,很少 產業 像 汽車 產業 那樣 具有 獨特性 歷經 百餘 年 的 演變 全球化 浪潮 的 ...,30,很少(20291) 產業(29756) 像(289494) 汽車(21028) 產業(297...,"{'High': 29, 'Low': 1}","{'Unknown': 8, '第5級': 4, '第1*級': 2, '第2級': 1, ...",4,Y,Y,N,N
914,此外，資訊、通訊產業在汽車的應用越來越廣泛，預估到2010年，車用電子占汽車成本的比重將會提...,此外 資訊 通訊 產業 在 汽車 的 應用 越來越 廣泛 預估 到 2010年 車用 電子 ...,42,此外(30247) 資訊(37848) 通訊(5667) 產業(29756) 在(32842...,"{'High': 41, 'Low': 1}","{'第5級': 10, '第4級': 3, '第1*級': 2, '第2級': 5, '第1...",4,Y,Y,N,N


In [29]:
edsplit_df.to_csv('../results/EDsplit/edsplit_parameters_example.csv', index = False)

## Concordance

In [30]:
from nltk.text import Text

In [31]:
target_words = ['難得', '畢竟', '的確', '難免', '總是', '有助於']
corpus = [item for sublist in edsplit_ws for item in sublist]
text = Text(corpus)
dfs = []

for word in target_words:

  con_list = text.concordance_list(word)
  right_word = [x.right[0] for x in con_list]
  left_word = [x.left[-1] for x in con_list]
  context = [x.left + [word] + x.right for x in con_list]
  context = [' '.join(x) for x in context]

  df = pd.DataFrame({'left_word': left_word,
                     'target_word': word,
                     'right_word': right_word,
                     'context': context})
  dfs.append(df)

In [32]:
concordance_df = pd.concat(dfs)
concordance_df.to_csv('../results/EDsplit/edsplit_concordance_df.csv', index = False)